<a href="https://colab.research.google.com/github/aminnemati96/amazon_food_review_score_prediction/blob/main/NLP_Autoenocder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.utils import np_utils
from nltk.corpus import stopwords
from tensorflow.keras import optimizers
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, RepeatVector, Dropout

Using TensorFlow backend.


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9617081828405195291
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40984969871
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4428133777157393675
physical_device_desc: "device: 0, name: Quadro RTX 8000, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [ ]:
dataset = pd.read_csv('NLP/final.csv')
dataset = dataset.astype({"Text": str, "Score": int})

In [ ]:
X, y = (dataset['Text'].values, dataset['Score'].values)

In [ ]:
num_words=2000# dict size
maxlen=20# length of each doc
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(X)# extracting features
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=maxlen, padding='post')

In [ ]:
embed_dim = 150
latent_dim = 128
batch_size = 64

In [ ]:
encoder_inputs = Input(shape=(maxlen,), name='Encoder-Input')
emb_layer = Embedding(num_words, embed_dim,input_length = maxlen,
                      name='Body-Word-Embedding', mask_zero=False)
x = emb_layer(encoder_inputs)
state_h = LSTM(latent_dim, name='Encoder-Last-LSTM')(x)
encoder_model = Model(inputs=encoder_inputs,
                      outputs=state_h, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(encoder_inputs)
decoded = RepeatVector(maxlen)(seq2seq_encoder_out)
decoder_LSTM = LSTM(latent_dim, return_sequences=True,
                    name='Decoder-LSTM-before')
decoder_LSTM_output = decoder_LSTM(decoded)
decoder_dense = Dense(num_words, activation='softmax',
                      name='Final-Output-Dense-before')
decoder_outputs = decoder_dense(decoder_LSTM_output)
seq2seq_Model = Model(encoder_inputs,decoder_outputs )
seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001),
                      loss='sparse_categorical_crossentropy')
history = seq2seq_Model.fit(X_pad, np.expand_dims(X_pad, -1),
                            batch_size=batch_size,
                            epochs=10,
                            validation_split=0.12, verbose=1)

Train on 500239 samples, validate on 68215 samples
Epoch 1/10
500239/500239 [==============================] - 92s 184us/sample - loss: 4.8612 - val_loss: 3.6599
Epoch 2/10
500239/500239 [==============================] - 87s 174us/sample - loss: 2.9755 - val_loss: 2.4649
Epoch 3/10
500239/500239 [==============================] - 86s 173us/sample - loss: 2.0684 - val_loss: 1.8411
Epoch 4/10
500239/500239 [==============================] - 86s 172us/sample - loss: 1.4498 - val_loss: 1.3092
Epoch 5/10
500239/500239 [==============================] - 86s 173us/sample - loss: 1.0101 - val_loss: 0.9858
Epoch 6/10
500239/500239 [==============================] - 86s 172us/sample - loss: 0.7257 - val_loss: 0.6269
Epoch 7/10
500239/500239 [==============================] - 86s 172us/sample - loss: 0.5199 - val_loss: 0.6033
Epoch 8/10
500239/500239 [==============================] - 85s 170us/sample - loss: 0.4018 - val_loss: 0.4407
Epoch 9/10
500239/500239 [==============================] - 8

In [ ]:
contents = tokenizer.texts_to_sequences(X)
contents = pad_sequences(contents, maxlen=maxlen, padding='post')
X_encoded = encoder_model.predict(contents)
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = np_utils.to_categorical(y)

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(X_encoded, y, test_size = 0.25, random_state = 1)

In [ ]:
model = Sequential()
model.add(Dense(1000, kernel_initializer='random_uniform',
                activation='relu', input_dim=128))
model.add(Dense(1000, kernel_initializer='random_uniform',
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, kernel_initializer='random_uniform',
                activation='relu'))
model.add(Dense(5, kernel_initializer='random_uniform',
                activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, 
          epochs=6, batch_size=100, verbose=1)

Train on 426340 samples
Epoch 1/6
426340/426340 [==============================] - 18s 41us/sample - loss: 1.0105 - accuracy: 0.6481
Epoch 2/6
426340/426340 [==============================] - 17s 40us/sample - loss: 0.9892 - accuracy: 0.6517
Epoch 3/6
426340/426340 [==============================] - 17s 40us/sample - loss: 0.9817 - accuracy: 0.6530
Epoch 4/6
426340/426340 [==============================] - 18s 43us/sample - loss: 0.9764 - accuracy: 0.6541
Epoch 5/6
426340/426340 [==============================] - 18s 42us/sample - loss: 0.9726 - accuracy: 0.6549
Epoch 6/6
426340/426340 [==============================] - 18s 41us/sample - loss: 0.9692 - accuracy: 0.6555


In [ ]:
model.evaluate(X_test, y_test, batch_size=100)

142114/142114 [==============================] - 9s 64us/sample - loss: 0.9665 - accuracy: 0.6569


[0.966504280455974, 0.6569092]

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.46      0.31      0.37     13243
           1       0.20      0.00      0.00      7422
           2       0.27      0.01      0.03     10580
           3       0.41      0.02      0.03     20068
           4       0.67      0.98      0.80     90801

    accuracy                           0.66    142114
   macro avg       0.40      0.26      0.25    142114
weighted avg       0.56      0.66      0.55    142114



In [ ]:
val_old = [0, 1, 2, 3, 4]
val_new = [0, 0, 1, 2, 2]
d = dict(zip(val_old, val_new))
Y_test2 = [d.get(e, e) for e in Y_test]
y_pred2 = [d.get(e, e) for e in y_pred]
print(classification_report(Y_test2, y_pred2))

              precision    recall  f1-score   support

           0       0.60      0.26      0.36     20665
           1       0.27      0.01      0.03     10580
           2       0.82      0.98      0.89    110869

    accuracy                           0.80    142114
   macro avg       0.56      0.42      0.43    142114
weighted avg       0.74      0.80      0.75    142114

